Attempting to create a simple program to handle a simple case of 2 body System

In [11]:
import numpy as np
import math
import random as rd

List is created with following parameters : [Mass, X position, Y position, X velocity, Y velocity, X acceleration, Y acceleration]
This will be modified constantly to run values. For each time value, Then recalculate g force between

In [12]:

#Gravitational Constant
def G() :
    return 6.6743*(10**-11)

# Mass of Earth in kg
def EarthMass() :
    return 6*(10**24)

# Earth Orbital Velocity in m/s
def EarthOrbitalVel() :
    return 29722.2222

# Mass of Sun in kg
def SolarMass() :
    return 330000 * 6 * (10 ** 24)

#Astronomical Units
def AU():
    return 149597871 * 1000



# initializes dictionary for starting particles
def initalparticle(dictionary : dict, id : int, mass : float, x_pos : float, y_pos : float, x_vel : float, y_vel : float):
    dictionary.update({id : [mass, x_pos, y_pos, x_vel, y_vel, 0, 0]})

# Finds vector for calulations - returns vector pointing from particle 1 to particle 2 - allows direct calculation for gforce components
def findvector(dictionary, particle_1, particle_2) :
    x_1 = dictionary.get(particle_1)[1]
    x_2 = dictionary.get(particle_2)[1]
    y_1 = dictionary.get(particle_1)[2]
    y_2 = dictionary.get(particle_2)[2]
    
    dx = x_2 - x_1
    dy = y_2 - y_1
    magnitude = (dx**2 + dy**2)**.5
    sine = math.asin(dy/magnitude)
    cosine = math.acos(dx/magnitude)
    if(sine >= 0) :
        theta = cosine
    elif(sine < 0) :
        theta = -cosine
    
    #print("vector for calculation", [magnitude, theta * 180/math.pi])
    return [magnitude, theta]

# Force of Gravity of particle_2 on particle_1
def gforce(dictionary : dict, particle_1 : int, particle_2 : int) :
    vector = findvector(dictionary, particle_1, particle_2)
    m_2 = dictionary.get(particle_2)[0]
    acceleration = G()*m_2/(vector[0])**2

    
    dictionary.get(particle_1)[5] += acceleration*math.cos(vector[1])
    dictionary.get(particle_1)[6] += acceleration*math.sin(vector[1])
    #print(acceleration*math.cos(vector[1]),acceleration*math.sin(vector[1]))

# Calculates all of the gforces for all particle interactions
def apply_gforce(dictionary : dict, number_of_particles : int) :
    for x in range(0,number_of_particles):
        for y in range (0,number_of_particles):
            if x != y:
                #print(x, "and", y)
                gforce(dictionary, x, y)


# Approximates new position values
def position(dictionary : dict, index : int , dt : float) :
    x_vel = dictionary.get(index)[3]
    y_vel = dictionary.get(index)[4]
    x_acc = dictionary.get(index)[5]
    y_acc = dictionary.get(index)[6]

    dictionary.get(index)[1] += .5 * x_acc * (dt ** 2) + x_vel * dt 
    dictionary.get(index)[2] += .5 * y_acc * (dt ** 2) + y_vel * dt 
    #print("X axis   ", .5 * x_acc * (dt ** 2),  x_vel * dt , "   Y axis  ", .5 * y_acc * (dt ** 2), y_vel * dt)

# Approximates new velocity values 
def velocity(dictionary : dict, index : int , dt : float) :
    x_acc = dictionary.get(index)[5]
    y_acc = dictionary.get(index)[6]

    #print("X_ axis  ", dictionary.get(index)[3], x_acc * dt, "  Y_axis  ", dictionary.get(index)[4], y_acc * dt)

    dictionary.get(index)[3] += x_acc * dt
    dictionary.get(index)[4] += y_acc * dt
    
    

# clears both x and y acceleration so algorithm continues to work
def reset_acc(dictionary : dict, index : int) :
    dictionary.get(index)[5] = 0
    dictionary.get(index)[6] = 0

# records x and y positions for later graphing - fills x_list and y_list with another list that contains respective positions for each particle
def record_pos(dictionary : dict, x_list : list, y_list : list) :
    temp_x = []
    temp_y = []
    
    for index in dictionary :
        temp_x.append(dictionary.get(index)[1])
        temp_y.append(dictionary.get(index)[2])
    
    x_list.append(temp_x)
    y_list.append(temp_y)




# moves forward time in simulation - order for position / velocity matters 
def model(dictionary : dict, dt : float, x_list : list, y_list : list) :
    for index in dictionary :
        position(dictionary, index, dt)
        velocity(dictionary, index, dt)
        reset_acc(dictionary, index)

# takes particle index and prints out cvs text input for desmos graphing (hopefully)
def csv(particle : int, x_list : int, y_list) :
    for index in range(0, len(x_list)):
        x_pos = x_list[index][particle]
        y_pos = y_list[index][particle]
        print (f"{x_pos}, {y_pos}")
    

    
    

Testing 5 body sun with 4 Earths at theta 0, pi/2, pi, 3pi/2

In [14]:
particleinfo = {}
x_list = []
y_list = []
number_of_particles = 2


# Initializes amount of particles  149597871 * 1000
initalparticle(particleinfo, 0, EarthMass(), AU(), 0, 0, EarthOrbitalVel())
#initalparticle(particleinfo, 1, EarthMass(), -AU(), 0, 0, -EarthOrbitalVel())
#initalparticle(particleinfo, 2, EarthMass(), 0, AU(), -EarthOrbitalVel(), 0)
#initalparticle(particleinfo, 3, EarthMass(), 0, -AU(), EarthOrbitalVel(), 0)
initalparticle(particleinfo, 1, SolarMass(), 0, 0, 0, 0)


#print("inital conditions      ", particleinfo)


# Runs program and records - in seconds
delta_t = 60 * 60
trials = 24 * 365 * 10

for index in range(trials):
    apply_gforce(particleinfo,number_of_particles)
    model(particleinfo, delta_t, x_list, y_list)
    if (index % (24 * 12) == 0) :
        record_pos(particleinfo, x_list, y_list)


#print("x_list         ", x_list)
#print("y_list         ", y_list)

for i in particleinfo :
    csv(i,x_list,y_list)
    print("\n\n")

149597832735.5836, 106999999.92
146413275599.29675, 30704399490.790962
137037755929.28212, 60005795740.24173
121867409512.55133, 86772237401.63513
101543586904.43773, 109872083290.95798
76925696485.6501, 128329033399.2854
49054714232.44968, 141363578463.94128
19108912683.670425, 148426057651.89352
-11646283173.663483, 149219876252.04825
-41912135811.659195, 143713869785.61734
-70411785325.26819, 132143286337.5134
-95944387204.91406, 114999370209.02512
-117435852161.4722, 93008040347.88043
-133984009773.83142, 67098639761.198715
-144896284706.66977, 38364162794.713264
-149718324706.30673, 8014725248.144972
-148252433284.54742, -22673687511.21531
-140565114397.8213, -52412986385.525955
-126983508459.94101, -79956978153.24487
-108080966709.9943, -104153345294.44287
-84652454627.11665, -123991471322.96626
-57680878177.42852, -138644178008.77
-28295776055.712936, -147501712046.23804
2273893050.031003, -150196646634.0162
32750923610.6233, -146618734390.25653
61862911430.84602, -136919148505.